In [1]:
import trimesh
import numpy as np 
import glob

import open3d as o3d

meshfiles = []
point_cloud_arrays = []
path_names = [
    "fulldata-stl/20148_région1.stl",
    "fulldata-stl/20183_région1.stl",
    "fulldata-stl/20187_région1.stl",
    "fulldata-stl/10204_région1.stl"
]

for file in glob.glob("fulldata-stl/*"):
    if file not in path_names:
        meshfiles.append(file)

for item in meshfiles:
    mesh = o3d.io.read_triangle_mesh(item)
    mesh.compute_vertex_normals()
    point_cloud = o3d.geometry.PointCloud(mesh.vertices)
    point_cloud_arrays.append(point_cloud)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def normalized(points):
  points_min = points.min(axis=0)
  points_max = points.max(axis=0)
  points_normalized = (points - points_min) / (points_max - points_min)
  points_normalized = 2 * (points_normalized - 0.5)  # Scale to [-1, 1]
  return points_normalized

In [3]:
def convert_points_to_3d(points, grid_size):
  grid = np.zeros((grid_size, grid_size, grid_size), dtype=np.float32)

  points_normalized = normalized(points)

  # Convert normalized points to voxel grid indices
  voxel_indices = ((points_normalized + 1) / 2 * (grid_size - 1)).astype(int)

  # Populate the grid
  for idx in voxel_indices:
      grid[tuple(idx)] = 1.0  # You can accumulate values or set to 1.0 if the voxel is occupied
  return grid

In [4]:
grid_size = 50
grid_3ds = []
for point_cloud in point_cloud_arrays:
  grid_3d = convert_points_to_3d(np.asarray(point_cloud.points), grid_size)
  grid_3ds.append(grid_3d)

In [5]:
np.save("fulldata_3d_without_4_points.npy", grid_3ds)

In [1]:
import open3d as o3d
import numpy as np

# Step 1: Load the STL file as a mesh
mesh = o3d.io.read_triangle_mesh("./fulldata-stl/10134_région4.stl")

# Step 2: Sample points on the mesh
# You can sample points uniformly or using Poisson disk sampling.
# Uniform sampling
point_cloud = mesh.sample_points_uniformly(number_of_points=5000)

point_cloud.colors = o3d.utility.Vector3dVector(np.tile([1, 0, 0], (len(point_cloud.points), 1)))

# Step 4: Create a visualization window and set point size
vis = o3d.visualization.Visualizer()
vis.create_window()

# Add the point cloud to the window
vis.add_geometry(point_cloud)

# Get the render options to change point size
render_option = vis.get_render_option()
render_option.point_size = 0.7  # Adjust the point size, smaller values for smaller points

# Step 5: Run the visualization and destroy the window after closing
vis.run()
vis.destroy_window()

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
